# User keyword query

    Input 1: user query keywords
    Input 2: condition--and, or

    Output: frequency of the keywords
    (1): How many times are the keywords mentioned? 這個關鍵字被提到多少次?
    (2): How many pieces of news contain (mention) the keywords?  有幾篇新聞提到這個關鍵字?

    First article: ['肺炎','疫情', '肺炎']
    Second article:['陳時中','指揮中心','肺炎','陳時中']

    肺炎: 
    (1) '肺炎' are mentioned three times.  ==> frequency is 3
    (2) Two pieces of news mention '肺炎'. ==> occurrence is 2

    陳時中: 
    (1) '陳時中' are mentioned two times.  ==> frequency is 2
    (2) One pieces of news mention '陳時中'. ==> occurrence is 1
    


# Step 0: Load preprocessed news dataset

In [90]:
import pandas as pd
from datetime import datetime, timedelta

In [91]:
df = pd.read_csv('./cna_news_preprocessed.csv',sep='|')

In [92]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,index.php?k=1_2025-03-27_1,2025-03-27,PC,多平台\n《餓狼傳說 City of the Wolves》足球界傳奇巨星「克里斯蒂亞諾‧羅...,株式會社 SNK 今日（3/27）宣布，足球界傳奇巨星「克里斯蒂亞諾‧羅納度」確定將以可遊玩...,暫無,暫無,"[('遊戲', 8), ('足球', 8), ('傳說', 7), ('角色', 6), (...","['株式會社', ' SNK ', '今日', '（', '3/27', '）', '宣布'...","['株式會社', '宣布', '足球界', '傳奇', '巨星', '克里斯蒂亞諾‧羅納度'...","[NerToken(word='株式會社', ner='ORG', idx=(0, 4)),...","[('株式會社', 'Nc'), (' SNK ', 'FW'), ('今日', 'Nd')...",https://gnn.gamer.com.tw/detail.php?sn=282906,https://p2.bahamut.com.tw/B/2KU/58/2fc0d3915aa...


# Step 1: Select news with user input keywords, category, duration

## (1)Improved Search from "content" column

In [93]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: keywords 
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in user_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in user_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


## (2)Search keywords from "content" column (Another way)

In [94]:
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText_v0(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        df_query = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]

    return df_query


## (3) Search from token_v2 (Another way)

In [95]:
# Searching keywords from "token_v2" column
def filter_dataFrame_v0(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        query_df = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        query_df = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        query_df = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        query_df = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    return query_df

In [96]:
user_keywords=['烏克蘭','俄羅斯']
cond='and'
cate='全部'
weeks=4
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

0

In [97]:
user_keywords=['烏克蘭','俄羅斯']
cond='and'
cate='國際'
weeks=4
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

0

In [98]:
user_keywords=['烏克蘭','俄羅斯']
cond='or'
cate='政治'
weeks=4
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

0

In [99]:
df_query.head()

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link


# Step 2: calculate frequency and occurence

In [100]:
# For the query_df, count the occurence and frequency for each category.

# (1) cate_occurence={}  被多少篇新聞報導 How many pieces of news contain the keywords.
# (2) cate_freq={}       被提到多少次? How many times are the keywords mentioned


news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']

def count_keyword(query_df, user_keywords):
    cate_occurence={}
    cate_freq={}

    for cate in news_categories:
        cate_occurence[cate]=0
        cate_freq[cate]=0

    for idx, row in query_df.iterrows():
        # count number of news
        cate_occurence[row.category] += 1
        cate_occurence['全部'] += 1
        
        # count user keyword frequency by checking every word in tokens_v2
        tokens = eval(row.tokens_v2)
        freq =  len([word for word in tokens if (word in user_keywords)])
        cate_freq[row.category] += freq
        cate_freq['全部'] += freq
        
    return cate_freq, cate_occurence

In [101]:
user_keywords=['烏克蘭','俄羅斯']
cond='or'
cate='全部'
weeks=4
# Step 1 fitering data
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

# Step 2: calculating frequency and occurence
count_keyword(df_query, user_keywords)

({'政治': 0,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 0},
 {'政治': 0,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 0})

In [102]:
len(df_query)

0

# Demonstrate step by step

How many news are related to "烏克蘭" ?

    How many pieces of news mentioned "烏克蘭"
    How many pieces of news are related to "烏克蘭"?

    You can calculate and get the answer from the following fields: tokens, tokens_v2, or content. (Get very similar results)

    We use "tokens_v2" because it contains only some important keywords which were selected in the pre-process step.

    
A flexible appraoch for And OR condiction

        User all()  any()
        df = pd.DataFrame({'col': ["apple is delicious",
                                "banana is delicious",
                                "apple and banana both are delicious"]})

        targets = ['apple', 'banana']

        # Any word from `targets` are present in sentence.
        >>> df.col.apply(lambda sentence: any(word in sentence for word in targets))
        0    True
        1    True
        2    True
        Name: col, dtype: bool

        # All words from `targets` are present in sentence.
        >>> df.col.apply(lambda sentence: all(word in sentence for word in targets))
        0    False
        1    False
        2     True
        Name: col, dtype: bool

## and &, or | 

In [103]:
True & True

True

In [104]:
True & False

False

In [105]:
True | True

True

In [106]:
True | False

True

## "in" is very powerful in Python!

#### in a string

In [107]:
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'

In [108]:
'勸阻官兵' in text

True

In [109]:
'延燒，國防部' in text

True

In [110]:
'台灣' in text

False

In [111]:
'烏克蘭' in text

True

In [112]:
# & and
('台灣' in text)  & ('烏克蘭' in text)

False

In [113]:
# & and
('台灣' in text) and ('烏克蘭' in text)

False

In [114]:
('台灣' in text)  | ('烏克蘭' in text)

True

In [115]:
('台灣' in text)  or ('烏克蘭' in text)

True

In [116]:
# This is also a string.
text = "['武漢', '烏克蘭', '疫情', '全球', '延燒', '國防部', '疫情', '高風險', '國家', '地區', '官兵', '過境', '國防部', '政策', '全球', '國家', '地區', '轄下', '單位']"

In [117]:
'烏克蘭' in text

True

In [118]:
'台灣' in text

False

In [119]:
('台灣' in text)  & ('烏克蘭' in text)

False

In [120]:
('台灣' in text)  | ('烏克蘭' in text)

True

#### in a list

In [121]:
user_keyword=['烏克蘭','台灣']
'烏克蘭' in user_keyword

True

In [122]:
# Check out the first news
df.content[0]

'株式會社 SNK 今日（3/27）宣布，足球界傳奇巨星「克里斯蒂亞諾‧羅納度」確定將以可遊玩角色的身分在對戰格鬥遊戲新作《餓狼傳說 City of the Wolves》（PS5 / PS4 / Xbox Series X|S / PC）中登場！餓狼傳説 CotW × 克里斯蒂亞諾‧羅納度｜參戰發布影片1/3克里斯蒂亞諾‧羅納度 | Cristiano Ronaldo克里斯蒂亞諾‧羅納度在其職業生涯中贏得了 34 座獎杯，其中包括在英格蘭、西班牙和義大利贏得的 7 次聯賽冠軍、5 次歐洲冠軍聯賽冠軍、4 次國際足聯世俱杯冠軍、歐洲足球錦標賽冠軍和歐洲國家聯賽冠軍。他是有史以來最偉大的足球運動員之一。他還是歷史上第一位連續五次在世界盃上進球的球員。他還獲得過無數個人獎項，包括 5 次金球獎、4 次歐洲金靴獎和 2 次國際足聯最佳球員獎。他還是保持著男子足球史上正式比賽進球最多紀錄的球員，為俱樂部和國家隊打進了超過 900 個進球。克里斯蒂亞諾‧羅納度 | Cristiano Ronaldo立於世界最高峰的頂尖足球選手。在賽季休假期間，為磨練自身新的足球技巧而造訪南鎮。以足球創作出的各種各樣的招式，即便是格鬥家也無法抵擋。格鬥流派：足球 + 武術生日：2 月 5 日血型：不明珍視：足球、家人《餓狼傳說 City of the Wolves》概要《餓狼傳說 City of the Wolves》是 1991 年問世的 SNK 人氣對戰格鬥遊戲《餓狼傳說》系列時隔 26 年的最新作。搭載了刺激感性的獨特美術風格、加速戰鬥興奮度的「REV 系統」、以及更加進化的「戰鬥系統」。此外，以從初學者到熟練者都能充分享受的 2 種操作方式為首，還準備了各種新功能和新元素。以充滿慾望的南鎮為舞台，新的「傳說」即將開始。《餓狼傳說 City of the Wolves》商品資訊遊戲名稱：餓狼傳說 City of the Wolves遊戲原名：FATAL FURY: City of the Wolves遊戲類型：對戰格鬥對應平台：PlayStation 5 / PlayStation 4（數位版 / 實體版）※ PS4 版可升級為 PS5 版Xbox Series X|S / Steam / Epic Games Store（數位版）發售日期：2025 年 4 月 24 日建議售價：特別版

In [123]:
qk = '烏克蘭'
text = df.content[0]
qk in text

False

In [124]:
qk = '外交部'
text = df.content[0]
qk in text

False

In [125]:
qk = '台灣'
text = df.content[0]
qk in text

False

In [126]:
text = df.content[0]
('烏克蘭' in text) & ('外交部' in text )

False

### Another "in" in Python. It is used for "for" loop.

In [127]:
user_keywords = ['烏克蘭','外交部']
text = df.content[0]
[(qk in text) for qk in user_keywords]

[False, False]

In [128]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
[(qk in text) for qk in user_keywords]

[False, False]

### all() any() 

    How to perform logical opertion with several conditions? 如何針對很多項去做邏輯運算?

    all(): perform "and" logical opertion 
    any(): perform "or" logical opertion

In [129]:
all( [True, True, True] ) #　True & True

True

In [130]:
any( [True, True] )

True

In [131]:
all( [True, False] )

False

In [132]:
any( [True, False] )

True

In [133]:
user_keywords = ['烏克蘭','國防部']
[word for word in user_keywords]

['烏克蘭', '國防部']

In [134]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
[(word in text) for word in user_keywords]

[True, True]

In [135]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
[word in text for word in user_keywords] # () can be removed

[True, True]

In [136]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
all([word in text for word in user_keywords])

True

In [137]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
all(word in text for word in user_keywords) # square brackets [] can be removed

True

In [138]:
# Check out the first news
user_keywords = ['烏克蘭','外交部']
text = df.content[0]
print([(qk in text) for qk in user_keywords])
all((qk in text) for qk in user_keywords)

[False, False]


False

In [139]:
user_keywords = ['烏克蘭','外交部']
text = df.content[0]
any((qk in text) for qk in user_keywords)

False

In [140]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
[(qk in text) for qk in user_keywords]

[False, False]

In [141]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
all((qk in text) for qk in user_keywords)

False

In [142]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
any((qk in text) for qk in user_keywords)

False

## Using apply() and lambda function

How to check out keyword occurency for every news?

In [143]:
# Use apply() and lambda function
user_keywords = ['烏克蘭','外交部']
df.content.apply(lambda text: all([(qk in text) for qk in user_keywords]))

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
Name: content, dtype: bool

In [144]:
user_keywords = ['烏克蘭','外交部']

In [145]:
[qk for qk in user_keywords]

['烏克蘭', '外交部']

In [146]:
text = df.content[0]
text

'株式會社 SNK 今日（3/27）宣布，足球界傳奇巨星「克里斯蒂亞諾‧羅納度」確定將以可遊玩角色的身分在對戰格鬥遊戲新作《餓狼傳說 City of the Wolves》（PS5 / PS4 / Xbox Series X|S / PC）中登場！餓狼傳説 CotW × 克里斯蒂亞諾‧羅納度｜參戰發布影片1/3克里斯蒂亞諾‧羅納度 | Cristiano Ronaldo克里斯蒂亞諾‧羅納度在其職業生涯中贏得了 34 座獎杯，其中包括在英格蘭、西班牙和義大利贏得的 7 次聯賽冠軍、5 次歐洲冠軍聯賽冠軍、4 次國際足聯世俱杯冠軍、歐洲足球錦標賽冠軍和歐洲國家聯賽冠軍。他是有史以來最偉大的足球運動員之一。他還是歷史上第一位連續五次在世界盃上進球的球員。他還獲得過無數個人獎項，包括 5 次金球獎、4 次歐洲金靴獎和 2 次國際足聯最佳球員獎。他還是保持著男子足球史上正式比賽進球最多紀錄的球員，為俱樂部和國家隊打進了超過 900 個進球。克里斯蒂亞諾‧羅納度 | Cristiano Ronaldo立於世界最高峰的頂尖足球選手。在賽季休假期間，為磨練自身新的足球技巧而造訪南鎮。以足球創作出的各種各樣的招式，即便是格鬥家也無法抵擋。格鬥流派：足球 + 武術生日：2 月 5 日血型：不明珍視：足球、家人《餓狼傳說 City of the Wolves》概要《餓狼傳說 City of the Wolves》是 1991 年問世的 SNK 人氣對戰格鬥遊戲《餓狼傳說》系列時隔 26 年的最新作。搭載了刺激感性的獨特美術風格、加速戰鬥興奮度的「REV 系統」、以及更加進化的「戰鬥系統」。此外，以從初學者到熟練者都能充分享受的 2 種操作方式為首，還準備了各種新功能和新元素。以充滿慾望的南鎮為舞台，新的「傳說」即將開始。《餓狼傳說 City of the Wolves》商品資訊遊戲名稱：餓狼傳說 City of the Wolves遊戲原名：FATAL FURY: City of the Wolves遊戲類型：對戰格鬥對應平台：PlayStation 5 / PlayStation 4（數位版 / 實體版）※ PS4 版可升級為 PS5 版Xbox Series X|S / Steam / Epic Games Store（數位版）發售日期：2025 年 4 月 24 日建議售價：特別版

In [147]:
[(qk in text) for qk in user_keywords]

[False, False]

In [148]:
# 
all([(qk in text) for qk in user_keywords])

False

In [149]:
# Square brackets can be removed
all((qk in text) for qk in user_keywords)

False

In [150]:
df[df.tokens_v2.apply(lambda text: all([word in text for word in user_keyword]))]

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link


In [151]:
len(df[df.tokens_v2.apply(lambda text: all([word in text for word in user_keyword]))])

0

### lambda: how does it work?
lambda is a function with only one line

In [152]:
func = lambda x: x+5  # parameters函數的參數   func(55, 175)  func(x) -> x+5

In [153]:
func(2)

7

In [154]:
numbers=[1,2,3]

In [155]:
[func(x) for x in numbers]

[6, 7, 8]

In [156]:
numbers=[1,2,3]
list(map(func, numbers))

[6, 7, 8]

In [157]:
numbers=[1,2,3]
list(map(lambda x: x+5, numbers))

[6, 7, 8]

In [158]:
def func2(x):
    return x+10

numbers=[1,2,3]
list(map(func2, numbers))

[11, 12, 13]

### How to use pandas apply()?

In [159]:
df_test = pd.DataFrame([1,2,3], columns=['number'])

In [160]:
df_test

,number
0,1
1,2
2,3


In [161]:
df_test.number.apply(lambda x: x+5) # add 5 for each element in df_test.number

0    6
1    7
2    8
Name: number, dtype: int64

In [162]:
df.tokens_v2.apply(lambda text: all([word in text for word in user_keyword]))

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
Name: tokens_v2, dtype: bool

In [163]:
df.content.apply(lambda text: all([word in text for word in user_keyword]))

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
Name: content, dtype: bool

In [164]:
user_keywords = ['烏克蘭','外交部']
def func3(text): 
    return all([word in text for word in user_keyword])

df.content.apply(func3)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
Name: content, dtype: bool

# Function of fitering news

Put them all together!

In [165]:
# Searching keywords from "token_v2" column
def filter_dataFrame(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        query_df = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        query_df = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        query_df = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        query_df = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    return query_df

### For "politics" category 政治類

In [166]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='政治'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

In [167]:
# end date: the date of the latest record of news
end_date = df.date.max()

# start date
start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

In [168]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='政治'
weeks=2

In [169]:
(df.category == cate) & ( (df['date'] >= start_date) & (df['date'] <= end_date)  ) & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
dtype: bool

In [170]:
query_df = df[(
    df.category == cate)   # category
    & (df.date >= start_date) & (df.date <= end_date) # duration
    & df.tokens_v2.apply(lambda row: all((qk in row) for qk in user_keywords)) # user keywords
    ]
len(query_df)

0

In [171]:
query_df

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link


### For "全部""All" category (不必篩選類別!)

In [172]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='全部'
weeks=2

In [173]:
if (cate == "全部") & (cond == 'and'):
    query_df = df[
        (df.date >= start_date) & (df.date <= end_date) 
        & df.tokens_v2.apply(lambda row: all((qk in row) for qk in user_keywords))
        ]
elif (cate == "全部") & (cond == 'or'):
    query_df = df[
        (df['date'] >= start_date) & (df['date'] <= end_date) 
        & df.tokens_v2.apply(lambda row: any((qk in row) for qk in user_keywords))]

In [174]:
len(query_df)

0

# Function of count_keyword() to calculate frequency and occurence

In [175]:
# For the query_df, count the occurence and frequency for each category.

# (1) cate_occurence={}  被多少篇新聞報導
# (2) cate_freq={}       被提到多少次?How many times were the keywords mentioned


news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']

def count_keyword(query_df, user_keywords):
    cate_occurence={}
    cate_freq={}

    for cate in news_categories:
        cate_occurence[cate]=0
        cate_freq[cate]=0

    for idx, row in query_df.iterrows():
        # count number of news
        cate_occurence[row.category] += 1 # add 1 to its category's occurence
        cate_occurence['全部'] += 1
        
        # count user keyword frequency by checking every word in tokens_v2
        tokens = eval(row.tokens_v2) # count frequency
        freq =  len([word for word in tokens if (word in user_keywords)]) # how manay times?
        cate_freq[row.category] += freq
        cate_freq['全部'] += freq
        
    return cate_freq, cate_occurence

In [176]:
user_keywords=['烏克蘭','台灣']
count_keyword(df_query, user_keywords)

({'政治': 0,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 0},
 {'政治': 0,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 0})

## Show it step by step by yourself

In [178]:
user_keywords=['遊戲','英雄聯盟']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

# count user keyword frequency by checking every word in tokens_v2
tokens = eval(df_query.tokens_v2[0]) # count frequency
freq =  len([word for word in tokens if (word in user_keywords)]) # how manay times?

KeyError: 0

In [ ]:
df_query

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20220314_1,2022-03-14,政治,外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾,民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...,0.01,"['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...","[('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...","['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...","['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...","[NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...","[('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...",https://www.cna.com.tw/news/aipl/202203140364....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
11,aipl_20220314_12,2022-03-14,政治,賑濟烏克蘭善款 累計突破6億元,財團法人賑災基金會今天表示，賑濟烏克蘭專戶開設到今天下午4時10分，累計收到善款已突破新台幣...,0.00,"['收到紐約台灣商會捐入一筆3萬5703美元、折合新台幣約101萬元的捐款', '外交部2日...","[('基金會', 6), ('烏克蘭', 6), ('收到', 4), ('新台幣', 4)...","['財團', '法人', '賑災', '基金會', '今天', '表示', '，', '賑濟...","['財團', '法人', '賑災', '基金會', '賑濟', '烏克蘭', '專戶', '...","[NerToken(word='今天', ner='DATE', idx=(9, 11)),...","[('財團', 'Na'), ('法人', 'Na'), ('賑災', 'VA'), ('基...",https://www.cna.com.tw/news/aipl/202203140241....,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
140,aopl_20220314_1,2022-03-14,國際,澳洲擴大制裁俄羅斯 歐盟要凍結切爾西老闆資產,（中央社雪梨/布魯塞爾14日綜合外電報導）澳洲為懲罰俄羅斯侵略烏克蘭，今天再將33名俄國寡頭...,0.00,"['將阿布拉莫維奇加入制裁的俄羅斯富豪名單', '而歐盟各國代表將在今天開會通過這項措施',...","[('制裁', 8), ('俄羅斯', 7), ('俄國', 7), ('歐盟', 5), ...","['（', '中央社', '雪梨', '/', '布魯塞爾', '14日', '綜合', '...","['中央社', '雪梨', '布魯塞爾', '綜合', '外電', '澳洲', '懲罰', ...","[NerToken(word='中央社', ner='ORG', idx=(1, 4)), ...","[('（', 'PARENTHESISCATEGORY'), ('中央社', 'Nc'), ...",https://www.cna.com.tw/news/aopl/202203140376....,NaN
141,aopl_20220314_2,2022-03-14,國際,俄羅斯提核協議新要求 伊朗外長將赴莫斯科討論,關於伊朗核子協議的談判，數天前因為俄羅斯提出新要求而觸礁，伊朗外交部長阿布杜拉希安15日將赴...,0.00,['他已與俄羅斯外長拉夫羅夫（Sergei Lavrov）討論了正在奧地利維也納進行的伊朗核...,"[('伊朗', 11), ('協議', 9), ('談判', 6), ('外長', 6), ...","['關於', '伊朗', '核子', '協議', '的', '談判', '，', '數', ...","['伊朗', '核子', '協議', '談判', '俄羅斯', '提出', '要求', '伊...","[NerToken(word='伊朗', ner='GPE', idx=(2, 4)), N...","[('關於', 'P'), ('伊朗', 'Nc'), ('核子', 'Na'), ('協議...",https://www.cna.com.tw/news/aopl/202203140361....,NaN


In [ ]:
df_query.iloc[0]

item_id                                           aipl_20220314_1
date                                                   2022-03-14
category                                                       政治
title                                    外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾
content         民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...
sentiment                                                    0.01
summary         ['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...
top_key_freq    [('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...
tokens          ['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...
tokens_v2       ['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...
entities        [NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...
token_pos       [('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...
link            https://www.cna.com.tw/news/aipl/202203140364....
photo_link      https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
Name: 0, dtype: object

In [ ]:
row = df_query.iloc[0]

In [ ]:
row.tokens_v2

"['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長', '吳釗燮', '現場', '協助', '整理', '物資', '志工', '加油', '民眾', '表達', '外交部', '發布', '新聞稿', '外交部', '民間', '募集', '烏克蘭', '難民', '物資', '響應', '外交部', '物資', '民眾', '包裹', '收到', '物資', '品項', '醫療', '口罩', '毛毯', '女性', '衛生', '用品', '尿片', '餅乾', '募集', '活動', '外交部', '民眾', '現場', '分類', '整理', '志工', '外交部', '人員', '吳釗燮', '外交部', '地下', '停車場', '吳釗燮', '在場', '慈濟', '民間', '組織', '志工', '自發', '到場', '幫忙', '善心', '人士', '互動', '烏克蘭', '人士', '義工', '現場', '協助', '吳釗燮', '致意', '慰問', '外交部', '照片', '到場', '幫忙', '烏克蘭', '志工', '家鄉', '情勢', '網紅佳娜', '外交部', '物資', '民眾', '物品', '外交部', '網站', '清單', '物品', '衣物', '外交部', '物品', '物品', '拆封', '包裝', '有效期', '整理', '轉運', '困擾', '時間', '民眾', '清單', '藥品', '外交部', '外交部', '民眾', '援助', '烏克蘭', '人道', '物資', '志工', '幫忙', '貢獻', '外交部', '各界', '人士', '時間', '精神', '投入', '國際', '人道', '援助', '表達', '敬意']"

In [ ]:
eval(row.tokens_v2)

['民眾',
 '烏克蘭',
 '愛心',
 '物資',
 '外交部',
 '收到',
 '外交部長',
 '吳釗燮',
 '現場',
 '協助',
 '整理',
 '物資',
 '志工',
 '加油',
 '民眾',
 '表達',
 '外交部',
 '發布',
 '新聞稿',
 '外交部',
 '民間',
 '募集',
 '烏克蘭',
 '難民',
 '物資',
 '響應',
 '外交部',
 '物資',
 '民眾',
 '包裹',
 '收到',
 '物資',
 '品項',
 '醫療',
 '口罩',
 '毛毯',
 '女性',
 '衛生',
 '用品',
 '尿片',
 '餅乾',
 '募集',
 '活動',
 '外交部',
 '民眾',
 '現場',
 '分類',
 '整理',
 '志工',
 '外交部',
 '人員',
 '吳釗燮',
 '外交部',
 '地下',
 '停車場',
 '吳釗燮',
 '在場',
 '慈濟',
 '民間',
 '組織',
 '志工',
 '自發',
 '到場',
 '幫忙',
 '善心',
 '人士',
 '互動',
 '烏克蘭',
 '人士',
 '義工',
 '現場',
 '協助',
 '吳釗燮',
 '致意',
 '慰問',
 '外交部',
 '照片',
 '到場',
 '幫忙',
 '烏克蘭',
 '志工',
 '家鄉',
 '情勢',
 '網紅佳娜',
 '外交部',
 '物資',
 '民眾',
 '物品',
 '外交部',
 '網站',
 '清單',
 '物品',
 '衣物',
 '外交部',
 '物品',
 '物品',
 '拆封',
 '包裝',
 '有效期',
 '整理',
 '轉運',
 '困擾',
 '時間',
 '民眾',
 '清單',
 '藥品',
 '外交部',
 '外交部',
 '民眾',
 '援助',
 '烏克蘭',
 '人道',
 '物資',
 '志工',
 '幫忙',
 '貢獻',
 '外交部',
 '各界',
 '人士',
 '時間',
 '精神',
 '投入',
 '國際',
 '人道',
 '援助',
 '表達',
 '敬意']

In [ ]:
tokens

['民眾',
 '烏克蘭',
 '愛心',
 '物資',
 '外交部',
 '收到',
 '外交部長',
 '吳釗燮',
 '現場',
 '協助',
 '整理',
 '物資',
 '志工',
 '加油',
 '民眾',
 '表達',
 '外交部',
 '發布',
 '新聞稿',
 '外交部',
 '民間',
 '募集',
 '烏克蘭',
 '難民',
 '物資',
 '響應',
 '外交部',
 '物資',
 '民眾',
 '包裹',
 '收到',
 '物資',
 '品項',
 '醫療',
 '口罩',
 '毛毯',
 '女性',
 '衛生',
 '用品',
 '尿片',
 '餅乾',
 '募集',
 '活動',
 '外交部',
 '民眾',
 '現場',
 '分類',
 '整理',
 '志工',
 '外交部',
 '人員',
 '吳釗燮',
 '外交部',
 '地下',
 '停車場',
 '吳釗燮',
 '在場',
 '慈濟',
 '民間',
 '組織',
 '志工',
 '自發',
 '到場',
 '幫忙',
 '善心',
 '人士',
 '互動',
 '烏克蘭',
 '人士',
 '義工',
 '現場',
 '協助',
 '吳釗燮',
 '致意',
 '慰問',
 '外交部',
 '照片',
 '到場',
 '幫忙',
 '烏克蘭',
 '志工',
 '家鄉',
 '情勢',
 '網紅佳娜',
 '外交部',
 '物資',
 '民眾',
 '物品',
 '外交部',
 '網站',
 '清單',
 '物品',
 '衣物',
 '外交部',
 '物品',
 '物品',
 '拆封',
 '包裝',
 '有效期',
 '整理',
 '轉運',
 '困擾',
 '時間',
 '民眾',
 '清單',
 '藥品',
 '外交部',
 '外交部',
 '民眾',
 '援助',
 '烏克蘭',
 '人道',
 '物資',
 '志工',
 '幫忙',
 '貢獻',
 '外交部',
 '各界',
 '人士',
 '時間',
 '精神',
 '投入',
 '國際',
 '人道',
 '援助',
 '表達',
 '敬意']

In [ ]:
[word for word in tokens if (word in user_keywords)]

['烏克蘭',
 '外交部',
 '外交部',
 '外交部',
 '烏克蘭',
 '外交部',
 '外交部',
 '外交部',
 '外交部',
 '烏克蘭',
 '外交部',
 '烏克蘭',
 '外交部',
 '外交部',
 '外交部',
 '外交部',
 '外交部',
 '烏克蘭',
 '外交部']

In [ ]:
len([word for word in tokens if (word in user_keywords)])

19

In [ ]:
freq

19

In [ ]:
cate_occurence={}
cate_freq={}

In [ ]:
cate_occurence

{}

In [ ]:
for cate in news_categories:
    cate_occurence[cate]=0
    cate_freq[cate]=0

In [ ]:
cate_occurence

{'政治': 0,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 0}

In [ ]:
cate_freq

{'政治': 0,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 0}

In [ ]:
cate_freq[row.category] += freq
cate_freq['全部'] += freq

In [ ]:
cate_freq

{'政治': 19,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 19}

In [ ]:
# count number of news
cate_occurence[row.category] += 1 # add 1 to its category's occurence
cate_occurence['全部'] += 1

In [ ]:
cate_occurence

{'政治': 1,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 1}

## Now we can conveniently call those functions

In [ ]:
user_keywords=['烏克蘭','台灣']
cond='or'
cate='全部'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

In [ ]:
len(df_query)

91

In [ ]:
count_keyword(df_query, user_keywords)

({'政治': 50,
  '科技': 37,
  '運動': 14,
  '證卷': 27,
  '產經': 34,
  '娛樂': 16,
  '生活': 7,
  '國際': 32,
  '社會': 8,
  '文化': 51,
  '兩岸': 11,
  '全部': 287},
 {'政治': 10,
  '科技': 11,
  '運動': 10,
  '證卷': 7,
  '產經': 9,
  '娛樂': 8,
  '生活': 5,
  '國際': 9,
  '社會': 5,
  '文化': 14,
  '兩岸': 3,
  '全部': 91})

In [ ]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='政治'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

In [ ]:
len(df_query)

2

In [ ]:
count_keyword(df_query, user_keyword)

({'政治': 14,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 14},
 {'政治': 2,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 2})

## Usage of apply, map (For reference)

In [ ]:
# apply usage
import pandas as pd

sample_df = pd.DataFrame({
    'Col 1': [3,4,5,6],
    'Col 2': [2,3,6,4],
    'Col 3': [8,8,9,8],

},index=["A","B","C","D"])
sample_df

,Col 1,Col 2,Col 3
A,3,2,8
B,4,3,8
C,5,6,9
D,6,4,8


In [ ]:
sample_df=sample_df.apply(lambda x: x+10)
sample_df

,Col 1,Col 2,Col 3
A,13,12,18
B,14,13,18
C,15,16,19
D,16,14,18


In [ ]:
sample_df["Col 1"]=sample_df["Col 1"].apply(lambda x: x-10)
sample_df

,Col 1,Col 2,Col 3
A,3,12,18
B,4,13,18
C,5,16,19
D,6,14,18
